In [1]:
import pandas as pd
import numpy as np


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
target = ["Max_Temperature","Min_Temperature","Rainfall"]
feature = ["Day","Month","Year"]


In [4]:
x_train, y_train = train[feature], train[target]
x_test, y_test = test[feature], test[target]


In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from xgboost import XGBRegressor



In [7]:
models = {
    "RandomForestRegressor":RandomForestRegressor(),
    "XGBRegressor":XGBRegressor()
}

param = {
    "RandomForestRegressor":{
        "n_estimators":[100,150],
        "criterion":["squared_error", "absolute_error", "friedman_mse", "poisson"],
        "max_depth":[1,3,5,7,9,12],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
    },
    "XGBRegressor":{
        "max_depth":[1,3,5,7,9],
        "learning_rate":[0.1,0.01,1,0.2],
        "n_estimators":[100,150],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2]
    }
}

In [8]:
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.metrics import mean_squared_error
# report = []
# for i in range(len(models)):
#     print("Enter")
#     print(list(models.keys())[i])
#     model = list(models.values())[i]
#     grid = RandomizedSearchCV(model,param_distributions=list(param.values())[i],scoring='neg_mean_absolute_error', n_jobs=-1,cv=3)
#     grid.fit(x_train, y_train)
#     y_pred = grid.predict(x_test)
#     print("MAE : ",mean_absolute_error(y_test,y_pred))
#     print("MSE : ",mean_squared_error(y_test,y_pred))
#     print("r2_score",r2_score(y_test,y_pred))
#     report.append(grid.best_params_)
#     print("Exit")
    
    
    

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
import tensorflow as tf
from sklearn.model_selection import GridSearchCV

In [10]:
X_train_lstm = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
X_test_lstm = x_test.reshape((x_test.shape[0], 1, x_train.shape[1]))
print(X_test_lstm)

[[[ 0.14707828 -0.12181955  1.7377312 ]]

 [[ 0.26073359 -0.12181955  1.7377312 ]]

 [[ 0.3743889  -0.12181955  1.7377312 ]]

 [[ 0.48804421 -0.12181955  1.7377312 ]]

 [[ 0.60169952 -0.12181955  1.7377312 ]]

 [[ 0.71535483 -0.12181955  1.7377312 ]]

 [[ 0.82901014 -0.12181955  1.7377312 ]]

 [[ 0.94266545 -0.12181955  1.7377312 ]]

 [[ 1.05632076 -0.12181955  1.7377312 ]]

 [[ 1.16997607 -0.12181955  1.7377312 ]]

 [[ 1.28363138 -0.12181955  1.7377312 ]]

 [[ 1.39728669 -0.12181955  1.7377312 ]]

 [[ 1.510942   -0.12181955  1.7377312 ]]

 [[ 1.62459731 -0.12181955  1.7377312 ]]

 [[-1.67140668  0.16783784  1.7377312 ]]

 [[-1.55775137  0.16783784  1.7377312 ]]

 [[-1.44409606  0.16783784  1.7377312 ]]

 [[-1.33044075  0.16783784  1.7377312 ]]

 [[-1.21678544  0.16783784  1.7377312 ]]

 [[-1.10313013  0.16783784  1.7377312 ]]

 [[-0.98947482  0.16783784  1.7377312 ]]

 [[-0.87581951  0.16783784  1.7377312 ]]

 [[-0.7621642   0.16783784  1.7377312 ]]

 [[-0.64850889  0.16783784  1.7377

In [11]:
model = Sequential([
    LSTM(150, return_sequences=True, input_shape=(1, X_train_lstm.shape[2])),
    Dropout(0.2),
    LSTM(100, return_sequences=True),
    Dropout(0.2),
    LSTM(50),
    Dense(3, activation='linear') 
])

d:\Data Science\Weather Predict\myenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [12]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 1, 150)         │        92,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 150)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 1, 100)         │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1, 100)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 50)             │        30,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │           153 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 223,153 (871.69 KB)

 Trainable params: 223,153 (871.69 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.fit(X_train_lstm, y_train, epochs=100, batch_size=20, validation_data=(X_test_lstm, y_test), verbose=1)

Epoch 1/100
265/265 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 405.7805 - mae: 15.3406 - val_loss: 150.4841 - val_mae: 7.6792
Epoch 2/100
265/265 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 94.6749 - mae: 7.1911 - val_loss: 97.3605 - val_mae: 5.0233
Epoch 3/100
265/265 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 46.8849 - mae: 4.9716 - val_loss: 86.7882 - val_mae: 4.2697
Epoch 4/100
265/265 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 38.6996 - mae: 4.3382 - val_loss: 86.0053 - val_mae: 4.1245
Epoch 5/100
265/265 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 35.5949 - mae: 4.2142 - val_loss: 86.6444 - val_mae: 4.0849
Epoch 6/100
265/265 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 35.8893 - mae: 4.1621 - val_loss: 86.4728 - val_mae: 4.1516
Epoch 7/100
265/265 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 34.1159 - mae: 4.1579 - val_loss: 81.1097 - val_mae: 3.6209
Epoch 8/100
265/265 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 27.0965 - mae: 3.5678 - val_loss: 80.8621 - val_mae: 3.5570
Epoch 9/100
265/265 ━━━━━━━━━

In [17]:
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_test_lstm)
print("MAE : ",mean_absolute_error(y_test,y_pred))
print("MSE : ",mean_squared_error(y_test,y_pred))
print("r2_score",r2_score(y_test,y_pred))

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
MAE :  2.3728578090667725
MSE :  68.06378936767578
r2_score 0.5214480757713318


In [18]:
data = np.array([[23,7,2024]])
data = scaler.transform(data)
data = data.reshape(1,1,data.shape[1])
pred = model.predict(data)
print(pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step


d:\Data Science\Weather Predict\myenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


[[34.56928   25.915081   2.6955872]]


In [16]:
model.save('model.h5')